In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Pivot subway data from long to wide format

In [133]:
#access subway data
#number of days in month * 143 unique stations = one month of entries
url_sc = 'https://data.cityofchicago.org/resource/5neh-572f.json?$where=date%20%3E%272021-05-15%27&$limit=4433'
subway_df_closed = pd.read_json(url_sc)

url_so = 'https://data.cityofchicago.org/resource/5neh-572f.json?$where=date%20%3E%272021-04-15%27&$limit=4290'
subway_df_open = pd.read_json(url_so)

In [134]:
subway_df_closed

# reshape from long to wide in pandas python

subway_df_closed_wide = subway_df_closed.pivot(index = 'date', columns='stationname', values='rides').reset_index()
subway_df_open_wide = subway_df_open.pivot(index = 'date', columns='stationname', values='rides').reset_index()

## Iteratively add SRI segements to dataframe

In [2]:
from datetime import date, timedelta
def dates(startyear, startmon, startday, endyear, endmon, endday): #in (year, month, day) format
    start_date = date(startyear, startmon, startday) 
    end_date = date(endyear, endmon, endday)    # perhaps date.now()

    delta = end_date - start_date   # returns timedelta
    days = []
    for i in range(delta.days + 1):
        day = start_date + timedelta(days=i)
        days.append(str(day))
    return days

In [3]:
days = dates(2018,3,5,2021,3,4)

In [4]:
#test behavior with one sample csv
df = pd.read_csv('/Users/amandakhoo/code/data4all-team6/data/SRI/1_SRI.csv')
maxtest = []
for day in days:
    maxtest.append(df.loc[df['time'].str.contains(day), 'SRI'].max())
    
testdf = pd.DataFrame()
testdf['date'] = days
testdf['SRI_max'] = maxtest
testdf

,date,SRI_max
0,2018-03-05,4.061097
1,2018-03-06,3.000957
2,2018-03-07,3.352608
3,2018-03-08,3.802497
4,2018-03-09,7.419558
...,...,...
1091,2021-02-28,2.815171
1092,2021-03-01,3.568193
1093,2021-03-02,2.350644
1094,2021-03-03,4.618797


In [ ]:
#run for all csvs in SRI folder (~over 1000 segments)
import os
import glob
  
# use glob to get all the csv files 
# in the folder
path = os.getcwd() + '/data/SRI/'
print(path) #this should be the same as the path to where the csvs are
print('/Users/amandakhoo/code/data4all-team6/data/SRI/')
csv_files = glob.glob(os.path.join(path, "*.csv"))
  
days = dates(2018,3,5,2021,3,4)
SRI = pd.DataFrame()
SRI['date'] = days
daily_max_SRIs = []
    
# loop over the list of csv files
for f in csv_files:
    # read the csv file
    df = pd.read_csv(f)
    
    #iterate over the days of interest
    for day in days:
        daily_max_SRI = df.loc[df['time'].str.contains(day), 'SRI'].max()
        daily_max_SRIs.append(daily_max_SRI)
    
    #add date column to the SRI df
    
    # print the location and filename
    #print('Location:', f)
    #print('File Name:', f.split("\\")[-1])
    
    #add new columns by segment ID with max SRI per day
    file_name = f.split("\\")[-1]
    segment = file_name.split('/')[-1].split('.')[0].split('_')[0] #get the segement number from the filename
    column_name = 'segment_' + segment + '_max_SRI'
    
    SRI[column_name] = daily_max_SRI
    
    
#pd.concat([SRI, df['SRI']], axis = 1)
    

/Users/amandakhoo/code/data4all-team6/data/SRI/
/Users/amandakhoo/code/data4all-team6/data/SRI/


/var/folders/5c/_dfm1tzd7xv8v4dgb843ng300000gn/T/ipykernel_31373/3126015583.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  SRI[column_name] = daily_max_SRI
/var/folders/5c/_dfm1tzd7xv8v4dgb843ng300000gn/T/ipykernel_31373/3126015583.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  SRI[column_name] = daily_max_SRI
/var/folders/5c/_dfm1tzd7xv8v4dgb843ng300000gn/T/ipykernel_31373/3126015583.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

In [ ]:
SRI.head()

## Combine SRI and Subway data
where the dates match

In [137]:
url = 'https://data.cityofchicago.org/resource/5neh-572f.json?$where=date%20%3E%272018-04-08%27&$limit=4433'
subway_df = pd.read_json(url)
subway_df_wide = subway_df.pivot(index = 'date', columns='stationname', values='rides').reset_index()

#change all date types to object
subway_df_wide['date'] = subway_df_wide['date'].dt.date.astype(str).values

In [139]:
dates = subway_df_wide['date'].values
dates

array(['2018-04-09', '2018-04-10', '2018-04-11', '2018-04-12',
       '2018-04-13', '2018-04-14', '2018-04-15', '2018-04-16',
       '2018-04-17', '2018-04-18', '2018-04-19', '2018-04-20',
       '2018-04-21', '2018-04-22', '2018-04-23', '2018-04-24',
       '2018-04-25', '2018-04-26', '2018-04-27', '2018-04-28',
       '2018-04-29', '2018-04-30', '2018-05-01', '2018-05-02',
       '2018-05-03', '2018-05-04', '2018-05-05', '2018-05-06',
       '2018-05-07', '2018-05-08', '2018-05-09'], dtype=object)

In [140]:
SRI_subway_subset = SRI[SRI['date'].isin(dates)]

In [141]:
print(SRI_subway_subset.dtypes)
print(subway_df_wide.dtypes)

date               object
segment_37_SRI    float64
segment_27_SRI    float64
segment_55_SRI    float64
segment_45_SRI    float64
                   ...   
segment_14_SRI    float64
segment_22_SRI    float64
segment_32_SRI    float64
segment_40_SRI    float64
segment_50_SRI    float64
Length: 91, dtype: object
stationname
date                    object
18th                   float64
35-Bronzeville-IIT     float64
35th/Archer            float64
43rd                   float64
                        ...   
Western-Cermak         float64
Western-Forest Park    float64
Western-Orange         float64
Western/Milwaukee      float64
Wilson                 float64
Length: 145, dtype: object


In [143]:
#merge
merged_subway_SRI_df = SRI_subway_subset.merge(subway_df_wide, on = 'date')

In [144]:
merged_subway_SRI_df.to_csv('data/SAMPLE_subway_SRI_data.csv')